In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv('aihu.csv')

In [3]:
df.head(4)
#0 - human 
#1 - ai

,text,generated
0,Car-free cities have become a subject of incre...,1
1,"Car Free Cities Car-free cities, a concept ga...",1
2,A Sustainable Urban Future Car-free cities ...,1
3,Pioneering Sustainable Urban Living In an e...,1


In [4]:
# df.rename(columns = {'Data':'text'}, inplace = True) 
df.rename(columns = {'generated':'class'}, inplace = True) 

In [5]:
# Check for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())

# Check for class balance
print("\nClass distribution:")
print(df['class'].value_counts())

# Basic preprocessing: removing missing values
df.dropna(inplace=True)

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['class'], test_size=0.2, random_state=42)

# Output the size of the splits
print("\nTraining set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])


Missing values in each column:
text     0
class    0
dtype: int64

Class distribution:
class
0    17508
1    11637
Name: count, dtype: int64

Training set size: 23316
Testing set size: 5829


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set maximum number of words to keep, based on word frequency
MAX_WORDS = 10000  
MAX_SEQUENCE_LENGTH = 250  

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences of integers
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

print("Shape of training data:", X_train_padded.shape)
print("Shape of testing data:", X_test_padded.shape)

2024-03-24 23:07:28.665769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 23:07:28.665821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 23:07:28.667740: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 23:07:28.679492: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Shape of training data: (23316, 250)
Shape of testing data: (5829, 250)


In [7]:
import numpy as np

# Calculate simple linguistic features
X_train_lengths = np.array([len(text.split()) for text in X_train])  # Text length
X_train_avg_word_length = np.array([np.mean([len(word) for word in text.split()]) for text in X_train])  # Average word length

X_test_lengths = np.array([len(text.split()) for text in X_test])
X_test_avg_word_length = np.array([np.mean([len(word) for word in text.split()]) for text in X_test])

# Stack features together for MLP input
X_train_features = np.vstack((X_train_lengths, X_train_avg_word_length)).T
X_test_features = np.vstack((X_test_lengths, X_test_avg_word_length)).T


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Model parameters
VOCAB_SIZE = min(MAX_WORDS, len(tokenizer.word_index) + 1)  # Vocabulary size
EMBEDDING_DIM = 100  # Size of the embedding vectors 
FILTERS = 128  # Number of filters for the Conv layer
KERNEL_SIZE = 5  # Size of the kernel for the Conv layer
DENSE_UNITS = 10  # Number of units in the Dense layer

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Dense
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Flatten

# CNN part for text
text_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
text_features = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(text_input)
text_features = Conv1D(FILTERS, KERNEL_SIZE, activation='relu')(text_features)
text_features = GlobalMaxPooling1D()(text_features)

# MLP part for linguistic features
linguistic_input = Input(shape=(X_train_features.shape[1],))
linguistic_features = Dense(DENSE_UNITS, activation='relu')(linguistic_input)

# Concatenate the outputs of the two models
combined_features = concatenate([text_features, linguistic_features])

# Final classification layer
classification_output = Dense(1, activation='sigmoid')(combined_features)

model = Model(inputs=[text_input, linguistic_input], outputs=classification_output)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 250)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 250, 100)             1000000   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 246, 128)             64128     ['embedding[0][0]']           
                                                                                                  
 input_2 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                              

2024-03-24 23:08:01.063719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9611 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5
2024-03-24 23:08:01.065404: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9491 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:09:00.0, compute capability: 7.5
2024-03-24 23:08:01.066811: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9491 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:87:00.0, compute capability: 7.5
2024-03-24 23:08:01.068142: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 7285 MB memory:  -> device: 3, name: NVIDIA GeForce GTX

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

# Convert labels to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True)

# Train the model
history = model.fit([X_train_padded, X_train_features], y_train, 
                    batch_size=32, 
                    epochs=10, 
                    validation_data=([X_test_padded, X_test_features], y_test),
                    callbacks=[early_stopping])

# Note: The batch_size and epochs are set to small numbers for demonstration and might need adjustment.

Epoch 1/10


2024-03-24 23:08:02.988110: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-24 23:08:02.991861: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-24 23:08:03.190260: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-24 23:08:04.438025: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f33916187a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-24 23:08:04.438079: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-03-24 23:08:04.438090: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-03-24 23:08:04.438098: I external/local_xla/xla/service/service.cc:176]   

729/729 [==============================] - 100s 130ms/step - loss: 0.7993 - accuracy: 0.9184 - val_loss: 0.0466 - val_accuracy: 0.9835
Epoch 2/10
729/729 [==============================] - 26s 36ms/step - loss: 0.0231 - accuracy: 0.9928 - val_loss: 0.0263 - val_accuracy: 0.9885
Epoch 3/10
729/729 [==============================] - 15s 21ms/step - loss: 0.0057 - accuracy: 0.9987 - val_loss: 0.0243 - val_accuracy: 0.9900
Epoch 4/10
729/729 [==============================] - 10s 14ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0238 - val_accuracy: 0.9902
Epoch 5/10
729/729 [==============================] - 8s 12ms/step - loss: 3.7724e-04 - accuracy: 1.0000 - val_loss: 0.0226 - val_accuracy: 0.9914
Epoch 6/10
729/729 [==============================] - 10s 14ms/step - loss: 1.6812e-04 - accuracy: 1.0000 - val_loss: 0.0225 - val_accuracy: 0.9911
Epoch 7/10
729/729 [==============================] - 10s 13ms/step - loss: 9.5565e-05 - accuracy: 1.0000 - val_loss: 0.0225 - val_accurac

183/183 [==============================] - 1s 3ms/step - loss: 0.0225 - accuracy: 0.9911
Test Accuracy: 99.11%


In [11]:
import matplotlib.pyplot as plt

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate([X_test_padded, X_test_features], y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

# Plotting training history
def plot_history(history):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

plot_history(history)

In [12]:
def predict_text(text):
    # Tokenize and pad the input text
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
    text_length = len(text.split())
    avg_word_length = np.mean([len(word) for word in text.split()])
    features = np.array([[text_length, avg_word_length]])
    
    # Make a prediction
    prediction = model.predict([padded_seq, features])
    
    # Interpret the prediction
    if prediction >= 0.5:
        print("The text is predicted to be an AI Generated Essay.")
    else:
        print("The text is predicted to be a Human Written Essay.")

# Example usage
user_input = input("Please enter a sentence to classify: ")
predict_text(user_input)


Please enter a sentence to classify:    I find it difficult to impose any sort of rigid cartographic principles on most of these. For example, the Fra Mauro map seems to duplicate labels liberally to show extent. There are 2 labels for Europe and 3 for Asia. Not to mention 2 Egypts, 2 Mesopotamias, 2 Tartarias etc.For the Waldseem√ºller map, the principle seems to be finding empty space, so Europe has the smallest letters of all the major land-masses, and the most dispersed (I also can't find the 'EU'). But like with Asia, Africa has been set in an essentially empty section of the map. It also has the province of Africa and Libya... so I'm with you on the problem of how to interpret 'Libya Inferior'. It seems to denote an intermediate sized region along the lines of Ethiopia. But we find duplications in medieval mappaemundi as well. The Ebstorf Map, for example, has Africa written in large capitals across the landmass, but also has 'Africa' written in small caps under the 'A' of Africa

1/1 [==============================] - 0s 324ms/step
The text is predicted to be a Human Written Essay.


In [13]:
user_input = input("Please enter a sentence to classify: ")
predict_text(user_input)

Please enter a sentence to classify:  Please enter a sentence to classify: Indiana University–Purdue University Indianapolis, commonly referred to as IUPUI,[a] is a public research university in Indianapolis, Indiana, United States. It is a collaboration between Indiana University and Purdue University that offers undergraduate, graduate, and professional degrees from both universities. Administered primarily through Indiana University as a core campus and secondarily through Purdue University as a regional campus, it is Indiana's primary urban research and academic health sciences institution. IUPUI is located in downtown Indianapolis along the White River and Fall Creek.  Among more than 550 degree programs, the urban university hosts the primary campuses for both the Indiana University School of Medicine, with more than 2,000 students, and the Indiana University School of Dentistry, the only dental school in the state. Also, the Indiana University Robert H. McKinney School of Law is

1/1 [==============================] - 0s 44ms/step
The text is predicted to be an AI Generated Essay.
